# Model evaluation with MLflow

now after completing data ingestion, validation, transformation and model trainer parts of the pipeline, we will start working on model evaluation and for that we will use MLflow

In [1]:
import os
os.chdir("../")

In [2]:
pwd

'd:\\projects\\MLflow_DL\\Machine-learning-project-with-MLflow'

we need to get mlflow tracking url so for that we will login into dagshub and connect our repository there


In [22]:
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/maymunashah/Machine-learning-project-with-MLflow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "maymunashah"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "261c3502f463e1c9a70faefb29e626aefd57b17"



In [36]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir:Path
    test_data_path:Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str




In [37]:
# now we need to create configuration manager function for data model_trainer task and for that we need the constants that store paths to config yaml schema and params and also read yaml and create directories functions to read them from utils

from mlProject.utils.common import read_yaml, create_directories, save_json
from mlProject.constants import *



class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
            schema_filepath = SCHEMA_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.schema = read_yaml(schema_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
          config = self.config.model_evaluation
          params = self.params.ElasticNet
          schema = self.schema.TARGET_COLUMN
          print(schema.name)

          create_directories([config.root_dir])

          model_evaluation_config = ModelEvaluationConfig(
                root_dir = config.root_dir,
                test_data_path = config.test_data_path,
                model_path = config.model_path,
                all_params = params,
                metric_file_name = config.metric_file_name,
                target_column = schema.name,
                mlflow_uri = "https://dagshub.com/maymunashah/Machine-learning-project-with-MLflow.mlflow"
          )
          return model_evaluation_config
          

In [38]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
import json




class ModelEvaluation:
    def __init__(self, config:ModelEvaluationConfig):
        self.config = config


    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_squared_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():
            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

In [39]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation  = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2025-01-23 15:03:44,375: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-23 15:03:44,379: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-23 15:03:44,381: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-23 15:03:44,383: INFO: common: created directory at: artifacts]
quality
[2025-01-23 15:03:44,384: INFO: common: created directory at: artifacts/model_evaluation]
[2025-01-23 15:04:03,573: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'RemoteDisconnected('Remote end closed connection without response')': /maymunashah/Machine-learning-project-with-MLflow.mlflow/api/2.0/mlflow/runs/create]
[2025-01-23 15:04:03,854: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/01/23 15:04:09 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'ElasticnetModel' already exists. Creating a new version of this model...
2025/01/23 15:04:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: ElasticnetModel, version 3
Created version '3' of model 'ElasticnetModel'.


🏃 View run honorable-squid-725 at: https://dagshub.com/maymunashah/Machine-learning-project-with-MLflow.mlflow/#/experiments/0/runs/0deefcfa908b4ed1ab69114346d4b404
🧪 View experiment at: https://dagshub.com/maymunashah/Machine-learning-project-with-MLflow.mlflow/#/experiments/0
